<a href="https://colab.research.google.com/github/KRiver28/prac/blob/master/chatbot_Tokenizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2022-04-12 06:55:05--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-04-12 06:55:05--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce23c3b2e22ae6c5b61c174fc34.dl.dropboxusercontent.com/cd/0/get/BjPmToV_1QXrQnwFYB8axiCuWElAK-z-Kz8Csx1bC91pxoLb6vYllU5RmmQKmV5ckXi5qVQuweTc7a_i_hSPCNz-vuwWDv5mY49kfQVK5FrYCbg0-3OQoXAsUVhVdoaQOEnzLgiR_XDtHqy8lBuAOMErIDWzsya8JyE_l9sGP_lyurrYZtn4MLsYUQ4zQuFFz_g/file?dl=1# [following]
--2022-04-12 06:55:06--  https://uce23c3b2e22ae6c5b61c174fc34.dl.dropboxusercontent.com/cd/0/get/

In [2]:
from konlpy.tag import Okt, Mecab
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re
import pickle
from tqdm.auto import tqdm

In [3]:
#############################
### 1. 파일 불러오기
#############################

data_path = '/content/drive/MyDrive/BIGDATA_STUDY/project_NLP/'

train_data = pd.read_csv(data_path + 'cafe_qa_train.csv',index_col = 0)
test_data = pd.read_csv(data_path + 'cafe_qa_validation.csv', index_col = 0)

#웹사이트, AS 없애기 (일단 카페 데이터만 해당)
condition = train_data[(train_data.Intent == '웹사이트') | (train_data.Intent == 'AS')].index
train_data.drop(condition, axis = 0, inplace = True)


data_frame = train_data



In [4]:
#################################
### 2. Intent 항목 레이블, 원핫인코딩(의도분석에 이용)
#################################

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#레이블 인코딩
lab_encoder = LabelEncoder()
lab_encoder.fit(data_frame['Intent'].unique())
lst = lab_encoder.classes_
label_list = dict(zip(lst, range(len(lst))))

print(label_list)    #원래값-숫자 목록

#레이블 데이터 적용
lab_in=lab_encoder.transform(data_frame['Intent'])   # label 값
ori_in=lab_encoder.inverse_transform(lab_in)              # 원래 값
data_frame['Intent'] = lab_in

#lable 값 저장(pkl파일로)
with open(data_path + 'intent_train.pkl','wb') as f:
    pickle.dump(data_frame['Intent'],f)  

#pkl파일 불러오기
with open(data_path + 'intent_train.pkl','rb') as f:
     intent = pickle.load(f)


#원핫 인코딩(선택)
data_frame = pd.get_dummies(data_frame, columns = ['Intent'])




{'결제': 0, '교환|반품|환불': 1, '매장': 2, '멤버십': 3, '배송': 4, '예약': 5, '제품': 6, '주문': 7, '포장': 8, '행사': 9}


In [5]:
#################################
### 3. 토크나이징(단어 분리)
#################################

stop_words = ['은', '는', '이', '가', '하', '아', '것', '들', '의', '있', '되', '수', '보', '주', '등', '한']
mecab = Mecab()
clean_train = []

def preprocessing(review, mecab, remove_stopwords = False, stop_words = []):
    review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]","", review) # 한글을 제외한 문자들 공백으로 치환
    word_review = mecab.morphs(review_text)

    if remove_stopwords:
        word_review = [token for token in word_review if not token in stop_words]

    return word_review

def cleaning(df):
    clean_data = []
    for i, review in tqdm(enumerate(df), total = len(df)):
        # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
        if type(review) == str:
            p = preprocessing(review, mecab, remove_stopwords=True, stop_words=stop_words)
            clean_data.append(p)
        else:
            clean_data.append([]) #string이 아니면 빈 값 추가
            print(i, review)

    return clean_data

def qa_tk(df):
  global data_path
  tk_q = cleaning(df['Q'])
  tk_a = cleaning(df['A'])
  with open(data_path + 'tk_q_train.pkl','wb') as f:
    pickle.dump(tk_q,f)
  with open(data_path + 'tk_a_train.pkl','wb') as f:
    pickle.dump(tk_a,f)
  return tk_q, tk_a

#토크나이징 하고 파일 저장하기
tk_q, tk_a = qa_tk(data_frame)
print(tk_q[0])
print(tk_a[0])


  0%|          | 0/9522 [00:00<?, ?it/s]

  0%|          | 0/9522 [00:00<?, ?it/s]

['케이크', '에서', '약간', '시큼', '맛', '나', '데', '상한', '건가요']
['죄송', '합니다', '새', '걸로', '바꿔', '드릴게요']


In [6]:
#######################################################
### 4. 임베딩(분리한 단어를 분석할 수 있게 벡터로 변환)
#######################################################

import itertools
def emb(data):
  word_len = [len(data_frame['Q'][i]) for i in data_frame.index]
  MAX_SEQ_LEN = int(sum(word_len)/len(word_len))            #한 문장당 단어 평균 갯수
  voc = set(list(itertools.chain.from_iterable(data)))  #단어 총 갯수
  
  VOCAB_SIZE=len(set(voc))
  tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token='<OOV>')
  tokenizer.fit_on_texts(data)          

  # 각 단어를 사전의 index로 표현
  train_seq = tokenizer.texts_to_sequences(data)
  

  #단어 사전 생성
  word2idx = {k:v for k, v in tokenizer.word_index.items() if v < VOCAB_SIZE}
  word2idx['<PAD>'] = 0

  
  x_train = pad_sequences(train_seq, maxlen=MAX_SEQ_LEN, padding='pre', truncating='pre')     # padding = 'pre' 하면 패딩값(0)이 앞에부터 채워짐.
  print('MAX_SEQ_LEN: ',MAX_SEQ_LEN)
  print('VOCAB_SIZE: ',VOCAB_SIZE)
  return x_train




In [7]:
#임베딩 하고 저장하기
q_train = emb(tk_q)
a_train = emb(tk_a)

print(q_train[0])
print(a_train[0])

with open(data_path + 'emb_q_train.pkl', 'wb') as f:
  pickle.dump(q_train, f)
with open(data_path + 'emb_a_train.pkl', 'wb') as f:
  pickle.dump(q_train, f)  

MAX_SEQ_LEN:  24
VOCAB_SIZE:  3850
MAX_SEQ_LEN:  24
VOCAB_SIZE:  3611
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0   19   60 1426  986   81  157  153  662   36]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  45   8 452 119 182  15]


#의도분석 모델

In [8]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
import pickle
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
from tensorflow.keras.optimizers import Adam

In [12]:
# # 데이터 읽어오기
# # train_file = data_path + 'cafe_qa_train.csv'
# # data = pd.read_csv(train_file, delimiter=',')
# queries = data['Q'].tolist()
# intents = data['Intent'].tolist()



In [15]:
# 학습용, 검증용, 테스트용 데이터셋 생성 ○3
# 학습셋:검증셋:테스트셋 = 7:2:1
ds = tf.data.Dataset.from_tensor_slices((q_train, lab_in))
ds = ds.shuffle(len(q_train))

train_size = int(len(q_train) * 0.7)
val_size = int(len(q_train) * 0.2)
test_size = int(len(q_train) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

In [19]:
# 하이퍼 파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = 7535 #전체 단어 개수
Intent_num = 10
MAX_SEQ_LEN = 24

In [20]:
# CNN 모델 정의  ○4
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

In [21]:
# 3,4,5gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(Intent_num, name='logits')(dropout_hidden)
predictions = Dense(Intent_num, activation=tf.nn.softmax)(logits)

In [22]:
# 모델 생성  ○5
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 24, 128)      964480      ['input_2[0][0]']                
                                                                                                  
 dropout_2 (Dropout)            (None, 24, 128)      0           ['embedding_1[0][0]']            
                                                                                                  
 conv1d_3 (Conv1D)              (None, 22, 128)      49280       ['dropout_2[0][0]']              
                                                                                              

In [23]:
# 모델 학습 ○6
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/5
334/334 [==============================] - 15s 12ms/step - loss: 1.3904 - accuracy: 0.5619 - val_loss: 0.6546 - val_accuracy: 0.8230
Epoch 2/5
334/334 [==============================] - 4s 11ms/step - loss: 0.6175 - accuracy: 0.8272 - val_loss: 0.3961 - val_accuracy: 0.8892
Epoch 3/5
334/334 [==============================] - 4s 11ms/step - loss: 0.4531 - accuracy: 0.8761 - val_loss: 0.2606 - val_accuracy: 0.9270
Epoch 4/5
334/334 [==============================] - 3s 10ms/step - loss: 0.3566 - accuracy: 0.9011 - val_loss: 0.1980 - val_accuracy: 0.9454
Epoch 5/5
334/334 [==============================] - 3s 10ms/step - loss: 0.3081 - accuracy: 0.9071 - val_loss: 0.1755 - val_accuracy: 0.9459


In [25]:
#모델 평가(테스트 데이터 셋 이용) ○7
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

48/48 [==============================] - 0s 9ms/step - loss: 0.1547 - accuracy: 0.9517
Accuracy: 95.168066
loss: 0.154713


In [26]:
# 모델 저장  ○8
model.save('intent_model.h5')